In [ ]:
import numpy as np
import pandas as pd
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
import torch
import tqdm

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
path = "/content/drive/MyDrive/Colab Notebooks/mbti_1.csv"
data = pd.read_csv(path)

data.head()

,type,posts
0,INFJ,'http://www.youtube.com/watch?v=qsXHcwe3krw|||...
1,ENTP,'I'm finding the lack of me in these posts ver...
2,INTP,'Good one _____ https://www.youtube.com/wat...
3,INTJ,"'Dear INTP, I enjoyed our conversation the o..."
4,ENTJ,'You're fired.|||That's another silly misconce...


In [ ]:
# Make all the data lowercase
data['posts'] = data['posts'].str.lower()
data.head()

,type,posts
0,INFJ,'http://www.youtube.com/watch?v=qsxhcwe3krw|||...
1,ENTP,'i'm finding the lack of me in these posts ver...
2,INTP,'good one _____ https://www.youtube.com/wat...
3,INTJ,"'dear intp, i enjoyed our conversation the o..."
4,ENTJ,'you're fired.|||that's another silly misconce...


In [ ]:
import re

In [ ]:
# Remove Hyperlinkes as they dont provide any information linked to classification
data['posts'] = data['posts'].apply(lambda x: re.sub(r'http\S+', '', x))
data.head()

,type,posts
0,INFJ,' and intj moments sportscenter not top ten...
1,ENTP,'i'm finding the lack of me in these posts ver...
2,INTP,"'good one _____ course, to which i say i k..."
3,INTJ,"'dear intp, i enjoyed our conversation the o..."
4,ENTJ,'you're fired.|||that's another silly misconce...


In [ ]:
# Set your OpenAI API key
# api_key = 'your-api-key'

# from openai import OpenAI
# client = OpenAI(api_key=api_key)

# # Function to obtain embeddings for each post using OpenAI's embedding model
# def get_embedding(text, model="text-embedding-3-small"):
#     text = text.replace("\n", " ")
#     return client.embeddings.create(input=[text], model=model).data[0].embedding

# Assuming 'df' is your DataFrame containing the data
# Function to separate posts based on the delimiter '|||'
def separate_posts(text):
    return text.replace("|||", ". ")

# Apply the function to separate posts
data['separate_posts'] = data['posts'].apply(separate_posts)

# Apply the function to obtain embeddings for each set of posts
# data['embeddings'] = data['separate_posts'].apply(lambda x: [get_embedding(post) for post in x])

# Flatten the list of embeddings for each index
# data_flattened = data.explode('embeddings')

# Store the embeddings in a CSV file
# data_flattened['embeddings'].apply(pd.Series).to_csv('embedded_1k_reviews.csv', index=False)


In [ ]:
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [ ]:
# Remove punctuation and special characters
data['posts'] = data['posts'].apply(lambda x: re.sub(r'[^\w\s]', '', x))

# Remove stop words
# stop_words = set(stopwords.words('english'))
# data['posts'] = data['posts'].apply(lambda x: ' '.join([word for word in x.split() if word not in stop_words]))

# # Lemmatize the words
# lemmatizer = WordNetLemmatizer()
# data['posts'] = data['posts'].apply(lambda x: ' '.join([lemmatizer.lemmatize(word) for word in x.split()]))

data.head()


,type,posts,separate_posts
0,INFJ,and intj moments sportscenter not top ten ...,' and intj moments sportscenter not top ten...
1,ENTP,im finding the lack of me in these posts very ...,'i'm finding the lack of me in these posts ver...
2,INTP,good one _____ course to which i say i kno...,"'good one _____ course, to which i say i k..."
3,INTJ,dear intp i enjoyed our conversation the oth...,"'dear intp, i enjoyed our conversation the o..."
4,ENTJ,youre firedthats another silly misconception t...,'you're fired.. that's another silly misconcep...


In [ ]:
from transformers import BertTokenizer

# Load the BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

# Create a function to tokenize a set of texts
def preprocessing_for_bert(data):
    """Perform required preprocessing steps for pretrained BERT.
    @param    data (np.array): Array of texts to be processed.
    @return   input_ids (torch.Tensor): Tensor of token ids to be fed to a model.
    @return   attention_masks (torch.Tensor): Tensor of indices specifying which
                  tokens should be attended to by the model.
    """
    # Create empty lists to store outputs
    input_ids = []
    attention_masks = []

    # For every sentence...
    for sent in tqdm.notebook.trange(len(data)):
        encoded_sent = tokenizer.encode_plus(
            text=data[sent],  # Preprocess sentence
            add_special_tokens=True,        # Add `[CLS]` and `[SEP]`
            max_length=MAX_LEN,                  # Max length to truncate/pad
            pad_to_max_length=True,
            return_attention_mask=True,      # Return attention mask
            )

        # Add the outputs to the lists
        input_ids.append(encoded_sent.get('input_ids'))
        attention_masks.append(encoded_sent.get('attention_mask'))

    # Convert lists to tensors
    input_ids = torch.tensor(input_ids)
    attention_masks = torch.tensor(attention_masks)

    return input_ids, attention_masks

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [ ]:
print(data['type'].unique())

['INFJ' 'ENTP' 'INTP' 'INTJ' 'ENTJ' 'ENFJ' 'INFP' 'ENFP' 'ISFP' 'ISTP'
 'ISFJ' 'ISTJ' 'ESTP' 'ESFP' 'ESTJ' 'ESFJ']


In [ ]:
from sklearn.model_selection import train_test_split



label2id = {
    'INFJ':0,
    'ENTP':1,
    'INTP':2,
    'INTJ':3,
    'ENTJ':4,
    'ENFJ':5,
    'INFP':6,
    'ENFP':7,
    'ISFP':8,
    'ISTP':9,
    'ISFJ':10,
    'ISTJ':11,
    'ESTP':12,
    'ESFP':13,
    'ESTJ':14,
    'ESFJ':15
}

X = data['separate_posts'].values
data['labels'] = data['type'].map(label2id)
y = data['labels'].values

X_train, X_test, y_train, y_test =\
    train_test_split(X, y, test_size=0.25, random_state=96)
X_test, X_val, y_test, y_val =\
    train_test_split(X_test, y_test, test_size=0.1, random_state=96)

In [ ]:
MAX_LEN = 64

# Print sentence 0 and its encoded token ids
token_ids = list(preprocessing_for_bert([X[0]])[0].squeeze().numpy())
print('Original: ', X[0])
print('Token IDs: ', token_ids)
print('Token IDs shape : ', len(token_ids))

# Run function `preprocessing_for_bert` on the train set and the validation set
print('Tokenizing data...')
train_inputs, train_masks = preprocessing_for_bert(X_train)
val_inputs, val_masks = preprocessing_for_bert(X_val)


  0%|          | 0/1 [00:00<?, ?it/s]

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


Original:  ' and intj moments    sportscenter not top ten plays    pranks. what has been the most life-changing experience in your life?.      on repeat for most of today.. may the perc experience immerse you.. the last thing my infj friend posted on his facebook before committing suicide the next day. rest in peace~    enfj7. sorry to hear of your distress. it's only natural for a relationship to not be perfection all the time in every moment of existence. try to figure the hard times as times of growth, as.... 84389  84390     .... welcome and stuff..   game. set. match.. prozac, wellbrutin, at least thirty minutes of moving your legs (and i don't mean moving them while sitting in your same desk chair), weed in moderation (maybe try edibles as a healthier alternative.... basically come up with three items you've determined that each type (or whichever types you want to do) would more than likely use, given each types' cognitive functions and whatnot, when left by.... all things in mo

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2619: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


  0%|          | 0/6506 [00:00<?, ?it/s]

  0%|          | 0/217 [00:00<?, ?it/s]

In [ ]:
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

# Convert other data types to torch.Tensor



train_labels = torch.tensor(y_train)
val_labels = torch.tensor(y_val)

# For fine-tuning BERT, the authors recommend a batch size of 16 or 32.
batch_size = 32

# Create the DataLoader for our training set
train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

# Create the DataLoader for our validation set
val_data = TensorDataset(val_inputs, val_masks, val_labels)
val_sampler = SequentialSampler(val_data)
val_dataloader = DataLoader(val_data, sampler=val_sampler, batch_size=batch_size)

In [ ]:
%%time
import torch
import torch.nn as nn
from transformers import BertModel

# Create the BertClassfier class
class BertClassifier(nn.Module):
    """Bert Model for Classification Tasks.
    """
    def __init__(self, freeze_bert=False):
        """
        @param    bert: a BertModel object
        @param    classifier: a torch.nn.Module classifier
        @param    freeze_bert (bool): Set `False` to fine-tune the BERT model
        """
        super(BertClassifier, self).__init__()
        # Specify hidden size of BERT, hidden size of our classifier, and number of labels
        D_in, H, D_out = 768, 50, 16

        # Instantiate BERT model
        self.bert = BertModel.from_pretrained('bert-base-uncased')

        # Instantiate an one-layer feed-forward classifier
        self.classifier = nn.Sequential(
            nn.Linear(D_in, H),
            nn.ReLU(),
            #nn.Dropout(0.5),
            nn.Linear(H, D_out)
        )

        # Freeze the BERT model
        if freeze_bert:
            for param in self.bert.parameters():
                param.requires_grad = False

    def forward(self, input_ids, attention_mask):
        """
        Feed input to BERT and the classifier to compute logits.
        @param    input_ids (torch.Tensor): an input tensor with shape (batch_size,
                      max_length)
        @param    attention_mask (torch.Tensor): a tensor that hold attention mask
                      information with shape (batch_size, max_length)
        @return   logits (torch.Tensor): an output tensor with shape (batch_size,
                      num_labels)
        """
        # Feed input to BERT
        outputs = self.bert(input_ids=input_ids,
                            attention_mask=attention_mask)


        # Extract the last hidden state of the token `[CLS]` for classification task
        last_hidden_state_cls = outputs[0][:, 0, :]

        # Feed input to classifier to compute logits
        logits = self.classifier(last_hidden_state_cls)

        return logits

CPU times: user 72 µs, sys: 0 ns, total: 72 µs
Wall time: 77.2 µs


In [ ]:
from transformers import AdamW, get_linear_schedule_with_warmup

def initialize_model(epochs=4):
    """Initialize the Bert Classifier, the optimizer and the learning rate scheduler.
    """
    # Instantiate Bert Classifier
    bert_classifier = BertClassifier(freeze_bert=False)

    # Tell PyTorch to run the model on GPU
    bert_classifier.to(device)

    # Create the optimizer
    optimizer = AdamW(bert_classifier.parameters(),
                      lr=5e-5,    # Default learning rate
                      eps=1e-8    # Default epsilon value
                      )

    # Total number of training steps
    total_steps = len(train_dataloader) * epochs

    # Set up the learning rate scheduler
    scheduler = get_linear_schedule_with_warmup(optimizer,
                                                num_warmup_steps=0, # Default value
                                                num_training_steps=total_steps)
    return bert_classifier, optimizer, scheduler

In [ ]:
import random
import time
# Specify loss function
loss_fn = nn.CrossEntropyLoss()
def set_seed(seed_value=42):
  """Set seed for reproducibility.
  """
  random.seed(seed_value)
  np.random.seed(seed_value)
  torch.manual_seed(seed_value)
  torch.cuda.manual_seed_all(seed_value)

def train(model, train_dataloader, val_dataloader=None, epochs=4, evaluation=False):
  """Train the BertClassifier model.
  """
  # Start training loop
  print("Start training...\n")
  for epoch_i in range(epochs):
    # =======================================
    #               Training
    # =======================================
    # Print the header of the result table
    print(f"{'Epoch':^7} | {'Batch':^7} | {'Train Loss':^12} | {'Val Loss':^10} | {'Val Acc':^9} | {'Elapsed':^9}")
    print("-"*70)
    # Measure the elapsed time of each epoch
    t0_epoch, t0_batch = time.time(), time.time()
    # Reset tracking variables at the beginning of each epoch
    total_loss, batch_loss, batch_counts = 0, 0, 0
    # Put the model into the training mode
    model.train()
    # For each batch of training data...
    for step, batch in enumerate(train_dataloader):
      batch_counts +=1
      # Load batch to GPU
      b_input_ids, b_attn_mask, b_labels = tuple(t.to(device) for t in batch)
      # print(f'b input ids shape : {b_input_ids.shape}')
      # print(f'b attn mask shape : {b_attn_mask.shape}')
      # Zero out any previously calculated gradients
      model.zero_grad()
      # Perform a forward pass. This will return logits.
      logits = model(b_input_ids, b_attn_mask)
      # Compute loss and accumulate the loss values
      loss = loss_fn(logits, b_labels)
      batch_loss += loss.item()
      total_loss += loss.item()
      # Perform a backward pass to calculate gradients
      loss.backward()
      # Clip the norm of the gradients to 1.0 to prevent "exploding gradients"
      torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
      # Update parameters and the learning rate
      optimizer.step()
      scheduler.step()
      # Print the loss values and time elapsed for every 20 batches
      if (step % 20 == 0 and step != 0) or (step == len(train_dataloader) - 1):
        # Calculate time elapsed for 20 batches
        time_elapsed = time.time() - t0_batch
        # Print training results
        print(f"{epoch_i + 1:^7} | {step:^7} | {batch_loss / batch_counts:^12.6f} | {'-':^10} | {'-':^9} | {time_elapsed:^9.2f}")
        # Reset batch tracking variables
        batch_loss, batch_counts = 0, 0
        t0_batch = time.time()

    # Calculate the average loss over the entire training data
    avg_train_loss = total_loss / len(train_dataloader)
    print("-"*70)
    # =======================================
    #               Evaluation
    # =======================================
    if evaluation == True:
      # After the completion of each training epoch, measure the model's performance
      # on our validation set.
      val_loss, val_accuracy = evaluate(model, val_dataloader)
      # Print performance over the entire training data
      time_elapsed = time.time() - t0_epoch
      print(f"{epoch_i + 1:^7} | {'-':^7} | {avg_train_loss:^12.6f} | {val_loss:^10.6f} | {val_accuracy:^9.2f} | {time_elapsed:^9.2f}")
      print("-"*70)
      print("\n")
  print("Training complete!")

def evaluate(model, val_dataloader):
  """After the completion of each training epoch, measure the model's performance
  on our validation set.
  """
  # Put the model into the evaluation mode. The dropout layers are disabled during
  # the test time.
  model.eval()
  # Tracking variables
  val_accuracy = []
  val_loss = []
  # For each batch in our validation set...
  for batch in val_dataloader:
    # Load batch to GPU
    b_input_ids, b_attn_mask, b_labels = tuple(t.to(device) for t in batch)
    # Compute logits
    with torch.no_grad():
      logits = model(b_input_ids, b_attn_mask)
      # Compute loss
      loss = loss_fn(logits, b_labels)
      val_loss.append(loss.item())
      # Get the predictions
      preds = torch.argmax(logits, dim=1).flatten()
      # Calculate the accuracy rate
      accuracy = (preds == b_labels).cpu().numpy().mean() * 100
      val_accuracy.append(accuracy)
      # Compute the average accuracy and loss over the validation set.
  val_loss = np.mean(val_loss)
  val_accuracy = np.mean(val_accuracy)
  return val_loss, val_accuracy

In [ ]:
set_seed(42)    # Set seed for reproducibility
device = 'cpu'
bert_classifier, optimizer, scheduler = initialize_model(epochs=2)
train(bert_classifier, train_dataloader, val_dataloader, epochs=2, evaluation=True)

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:429: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Start training...

 Epoch  |  Batch  |  Train Loss  |  Val Loss  |  Val Acc  |  Elapsed 
----------------------------------------------------------------------
   1    |   20    |   2.637477   |     -      |     -     |  401.30  
   1    |   40    |   2.438202   |     -      |     -     |  362.88  
   1    |   60    |   2.417989   |     -      |     -     |  363.44  
   1    |   80    |   2.353106   |     -      |     -     |  364.11  
   1    |   100   |   2.359498   |     -      |     -     |  355.19  
   1    |   120   |   2.287256   |     -      |     -     |  356.48  
   1    |   140   |   2.267804   |     -      |     -     |  360.87  
   1    |   160   |   2.253012   |     -      |     -     |  357.28  
   1    |   180   |   2.270497   |     -      |     -     |  357.43  
   1    |   200   |   2.193958   |     -      |     -     |  356.10  
   1    |   203   |   2.135375   |     -      |     -     |   41.72  
----------------------------------------------------------------------

In [ ]:
import torch.nn.functional as F
def bert_predict(model, test_dataloader):
  """
    Perform a forward pass on the trained BERT model to predict probabilities
    on the test set.
  """
  # Put the model into the evaluation mode. The dropout layers are disabled during
  # the test time.
  model.eval()
  all_logits = []
  # For each batch in our test set...
  for batch in test_dataloader:
    # Load batch to GPU
    b_input_ids, b_attn_mask = tuple(t.to(device) for t in batch)[:2]
    # Compute logits
    with torch.no_grad():
      logits = model(b_input_ids, b_attn_mask)
      all_logits.append(logits)
  # Concatenate logits from each batch
  all_logits = torch.cat(all_logits, dim=0)
  # Apply softmax to calculate probabilities
  probs = F.softmax(all_logits, dim=1).cpu().numpy()
  return probs

In [ ]:
# Compute predicted probabilities on the test set
probs = bert_predict(bert_classifier, val_dataloader)
# Evaluate the Bert classifier
evaluate_roc(probs, y_val)

NameError: name 'evaluate_roc' is not defined

In [ ]:
# Run `preprocessing_for_bert` on the test set
print('Tokenizing data...')
test_inputs, test_masks = preprocessing_for_bert(X_test)
# Create the DataLoader for our test set
test_dataset = TensorDataset(test_inputs, test_masks)
test_sampler = SequentialSampler(test_dataset)
test_dataloader = DataLoader(test_dataset, sampler=test_sampler, batch_size=32)

In [ ]:
# Compute predicted probabilities on the test set
probs = bert_predict(bert_classifier, test_dataloader)

In [ ]:
import matplotlib.pyplot as plt

# Distribution graphs (histogram/bar graph) of column data
def plotPerColumnDistribution(df, nGraphShown, nGraphPerRow):
    nunique = df.nunique()
    df = df[[col for col in df if nunique[col] > 1 and nunique[col] < 50]] # For displaying purposes, pick columns that have between 1 and 50 unique values
    nRow, nCol = df.shape
    columnNames = list(df)
    nGraphRow = (nCol + nGraphPerRow - 1) / nGraphPerRow
    plt.figure(num = None, figsize = (6 * nGraphPerRow, 8 * nGraphRow), dpi = 80, facecolor = 'w', edgecolor = 'k')
    for i in range(min(nCol, nGraphShown)):
        plt.subplot(nGraphRow, nGraphPerRow, i + 1)
        columnDf = df.iloc[:, i]
        if (not np.issubdtype(type(columnDf.iloc[0]), np.number)):
            valueCounts = columnDf.value_counts()
            valueCounts.plot.bar()
        else:
            columnDf.hist()
        plt.ylabel('counts')
        plt.xticks(rotation = 90)
        plt.title(f'{columnNames[i]} (column {i})')
    plt.tight_layout(pad = 1.0, w_pad = 1.0, h_pad = 1.0)
    plt.show()

plotPerColumnDistribution(data, 10, 5)

ValueError: Number of rows must be a positive integer, not 1.0

<Figure size 2400x640 with 0 Axes>